# Text Embedding with DJL Spark Support

In this example, we will use Jupyter Notebook to run Text Embedding with DJL Spark extension on Scala. To execute this Scala kernel successfully, you need to install [Almond](https://almond.sh/), a Scala kernel for Jupyter Notebook. Almond provide extensive functionalities for Scala and Spark applications.

[Almond installation instruction](https://almond.sh/docs/quick-start-install) (Note: only Scala 2.12 are tested)

After that, you can start with DJL's Scala notebook.


## Import dependencies

Firstly, let's import the depdendencies we need. We choose to use DJL TensorFlow as our backend engine.

In [ ]:
import $ivy.`org.apache.spark::spark-sql:3.3.2`
import $ivy.`ai.djl:api:0.22.1`
import $ivy.`ai.djl.spark:spark:0.22.1`
import $ivy.`ai.djl.pytorch:pytorch-model-zoo:0.22.1`
import $ivy.`ai.djl.pytorch:pytorch-native-cpu-precxx11:1.13.1`

Then we can import the packages we need to use. In the last two lines, we disabled the Spark logging in order to avoid polluting your cell outputs.

In [ ]:
import org.apache.spark.sql.NotebookSparkSession
import ai.djl.spark.task.text.TextEmbedder
import org.apache.spark.sql.SparkSession

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF) // avoid too much message popping out
Logger.getLogger("ai").setLevel(Level.OFF) // avoid too much message popping out

## Start Spark application

We can create a `NotebookSparkSession` through the Almond Spark plugin. It will internally apply all necessary jars to each of the worker node.

In [ ]:
// Create Spark session
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Let's create DataFrame with text values using Spark library:

In [ ]:
val df = spark.createDataFrame(Seq(
  (1, "Hello, y'all! How are you?"),
  (2, "Hello to you too!"),
  (3, "I'm fine, thank you!")
)).toDF("id", "text")
df.show(truncate=false)

Then we can run inference on the text. All we need to do is to create a `TextEmbedder` and run inference with DJL.

In [ ]:
val embedder = new TextEmbedder()
  .setInputCol("text")
  .setOutputCol("embedding")
  .setEngine("PyTorch")
  .setModelUrl("djl://ai.djl.huggingface.pytorch/sentence-transformers/all-MiniLM-L6-v2")
val outputDf = embedder.embed(df)
outputDf.show()